# This notebook is part 3 of using autogen with local ai models to query an arbitrary database.
In this notebook I will use autogen with ChatGPT to connect to a local database and query it.  I will not be using local AI models
I'm using this reference https://github.com/disler/multi-agent-postgres-data-analytics/tree/v1-prompt-engineering-an-entire-codebase

In [2]:
%pip install pyodbc
%pip install python-dotenv
%pip install pyautogen
%pip install --upgrade --force-reinstall openai==0.28.1

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
  Using cached openai-0.28.1-py3-none-any.whl.metadata (11 kB)
  Using cached requests-2.31.0-py3-none-any.whl.metadata (4.6 kB)
  Using cached tqdm-4.66.1-py3-none-any.whl.metadata (57 kB)
  Using cached aiohttp-3.9.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (7.4 kB)
  Using cached charset_normalizer-3.3.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (33 kB)
  Using cached urllib3-2.1.0-py3-none-any.whl.metadata (6.4 kB)
  Using cached certifi-2023.11.17-py3-none-any.whl.metadata (2.2 kB)
  Using cached attrs-23.1.0-py3-none-any.whl (61 kB)
  Using cached multidict-6.0.4-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (114 kB)
  Using cached yarl-1.9.4-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (31 kB)
  Using cached frozenlist-1.4.0-cp310-cp310-manylinux_2_5_x86_64.many

## Fetch the required environment variables

In [20]:
import os
import dotenv

dotenv.load_dotenv()

OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY")

## Create code to handle connecting to the database
The settings for the database should obviously be in a config file, but for now I'm just going to hard code them in.

In [34]:
# connect to MSSQL database
import pyodbc

SERVER = '127.0.0.1'
DATABASE = 'TimeBasedCommitments'
USERNAME = 'sa'
PASSWORD = 'BadDefaultPassword!'

connectionString = f'DRIVER={{ODBC Driver 17 for SQL Server}};SERVER={SERVER};DATABASE={DATABASE};UID={USERNAME};PWD={PASSWORD}'

conn = pyodbc.connect(connectionString)

# conn.close()

# Get all table definitions
The table information will be passed to the AI prompt so it can create queries.

In [22]:
# Get table names
get_all_tables_stmt = "SELECT TABLE_NAME FROM INFORMATION_SCHEMA.TABLES;"
cursor = conn.cursor()
cursor.execute(get_all_tables_stmt)
table_names = [row[0] for row in cursor.fetchall()]

# Get all table definitions and format them into CREATE TABLE statements
get_def_stmt = f"""
SELECT COLUMN_NAME, DATA_TYPE
FROM INFORMATION_SCHEMA.COLUMNS
WHERE TABLE_NAME = ?
"""

definitions = []
for table_name in table_names:
    cursor.execute(get_def_stmt, (table_name,))
    rows = cursor.fetchall()

    create_table_stmt = f"CREATE TABLE {table_name} (\n"
    for row in rows:
        create_table_stmt += f"{row[0]} {row[1]},\n"
    create_table_stmt = create_table_stmt.rstrip(",\n") + "\n);"
    
    definitions.append(create_table_stmt)

table_definitions = "\n\n".join(definitions)

## Define a function that agent can use to run SQL

In [23]:
def run_sql(sql):
    cursor = conn.cursor()
    cursor.execute(sql)
    rows = cursor.fetchall()
    print(f"\n\n-------- SQL QUERY RESULTS --------")
    print(rows)
    return rows

## Setup the AI model

In [25]:
import autogen

# build the gpt_configuration object
gpt4_config = {
    "use_cache": False,
    "temperature": 0,
    "config_list": autogen.config_list_from_models(["gpt-4"]),
    "request_timeout": 120,
    "functions": [
        {
            "name": "run_sql",
            "description": "Run a SQL query against the postgres database",
            "parameters": {
                "type": "object",
                "properties": {
                    "sql": {
                        "type": "string",
                        "description": "The SQL query to run",
                    }
                },
                "required": ["sql"],
            },
        }
    ],
}

# build the function map to make functions available to some autogen agents
function_map = {
    "run_sql": run_sql,
}

# create our terminate msg function
def is_termination_msg(content):
    have_content = content.get("content", None) is not None
    if have_content and "APPROVED" in content["content"]:
        return True
    return False

COMPLETION_PROMPT = "If everything looks good, respond with APPROVED"

## Create the agents

In [26]:
USER_PROXY_PROMPT = (
    "A human admin. Interact with the Product Manager to discuss the plan. Plan execution needs to be approved by this admin."
    + COMPLETION_PROMPT
)
DATA_ENGINEER_PROMPT = (
    "A Data Engineer. You follow an approved plan. Generate the initial SQL based on the requirements provided. Send it to the Sr Data Analyst to be executed."
    + COMPLETION_PROMPT
)
SR_DATA_ANALYST_PROMPT = (
    "Sr Data Analyst. You follow an approved plan. You run the SQL query, generate the response and send it to the product manager for final review."
    + COMPLETION_PROMPT
)
PRODUCT_MANAGER_PROMPT = (
    "Product Manager. Validate the response to make sure it's correct"
    + COMPLETION_PROMPT
)

## Configure the agent roles/settings

In [27]:
# create a set of agents with specific roles
# admin user proxy agent - takes in the prompt and manages the group chat
user_proxy = autogen.UserProxyAgent(
    name="Admin",
    system_message=USER_PROXY_PROMPT,
    code_execution_config=False,
    human_input_mode="NEVER",
    is_termination_msg=is_termination_msg,
)

# data engineer agent - generates the sql query
data_engineer = autogen.AssistantAgent(
    name="Engineer",
    llm_config=gpt4_config,
    system_message=DATA_ENGINEER_PROMPT,
    code_execution_config=False,
    human_input_mode="NEVER",
    is_termination_msg=is_termination_msg,
)

# sr data analyst agent - run the sql query and generate the response
sr_data_analyst = autogen.AssistantAgent(
    name="Sr_Data_Analyst",
    llm_config=gpt4_config,
    system_message=SR_DATA_ANALYST_PROMPT,
    code_execution_config=False,
    human_input_mode="NEVER",
    is_termination_msg=is_termination_msg,
    function_map=function_map,
)

# product manager - validate the response to make sure it's correct
product_manager = autogen.AssistantAgent(
    name="Product_Manager",
    llm_config=gpt4_config,
    system_message=PRODUCT_MANAGER_PROMPT,
    code_execution_config=False,
    human_input_mode="NEVER",
    is_termination_msg=is_termination_msg,
)

## Create the group chat and add the agents to the group

In [28]:
groupchat = autogen.GroupChat(
    agents=[user_proxy, data_engineer, sr_data_analyst, product_manager],
    messages=[],
    max_round=10,
)
manager = autogen.GroupChatManager(groupchat=groupchat, llm_config=gpt4_config)

## Prompt the group and initiate the chat

In [37]:
user_question = "how many users are in the database?"

prompt = f"Fulfill this database query: {user_question}. "

# add the database info to the prompt
POSTGRES_TABLE_DEFINITIONS_CAP_REF = "TABLE_DEFINITIONS"
prompt_suffix = f"Use these {POSTGRES_TABLE_DEFINITIONS_CAP_REF} to satisfy the database query."

prompt = f"""{prompt} {prompt_suffix}\n\n{POSTGRES_TABLE_DEFINITIONS_CAP_REF}\n\n{table_definitions}"""

# initiate the chat
result = user_proxy.initiate_chat(manager, clear_history=True, message=prompt)

print(result)

Admin (to chat_manager):

Fulfill this database query: how many users are in the database?.  Use these TABLE_DEFINITIONS to satisfy the database query.

TABLE_DEFINITIONS

CREATE TABLE tblAudit (
id bigint,
date datetime,
user_id bigint,
logdata nvarchar
);

CREATE TABLE tblCommitments (
id bigint,
active bit,
title varchar,
description varchar,
added datetime,
modified datetime,
instructions varchar,
frequency varchar,
lookahead int,
length int
);

CREATE TABLE tblServices (
id bigint,
service_name varchar,
version varchar,
last_connected datetime,
error_message varchar,
last_error datetime,
healthy bit
);

CREATE TABLE tblReports (
id int,
label varchar,
config nvarchar
);

CREATE TABLE tblReportSchedules (
id bigint,
schedule varchar,
report_name varchar,
last_sent datetime
);

CREATE TABLE tblResponsibleToCommitments (
id bigint,
commitment_id bigint,
user_id bigint
);

CREATE TABLE tblSources (
id bigint,
description varchar,
active bit
);

CREATE TABLE tblSourcesToCommitments (
i

## Cleanup

In [36]:
conn.close()

https://api.openai.com
